gridifyから転載し修正。必須6項目をそのまま保存する。

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
from andersan import tile
import datetime
import requests_cache
from retry_requests import retry
from logging import basicConfig, getLogger, INFO

OPENMETEO_ITEMS = (
    "temperature_2m",
    "weather_code",
    "cloud_cover",
    "wind_speed_10m",
    "pressure_msl",
    "shortwave_radiation",
)


def openmeteo_tiles(start_isodate, end_isodate, XY, zoom):
    logger = getLogger()
    # Setup the cache and retry mechanism
    # cache_session = requests_cache.CachedSession("airpollution", expire_after=3600)
    cache_session = requests_cache.CachedSession("airpollution")
    retry_session = retry(cache_session, retries=5, backoff_factor=0.2)

    lonlats = tile.lonlat(zoom, xy=XY)
    lons = lonlats[:, 0]
    lats = lonlats[:, 1]
    dts = datetime.datetime.fromisoformat(start_isodate)
    dte = datetime.datetime.fromisoformat(end_isodate)
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": ",".join([f"{x:.4f}" for x in lats]),
        "longitude": ",".join([f"{x:.4f}" for x in lons]),
        "hourly": ",".join(OPENMETEO_ITEMS),
        "start_date": dts.strftime("%Y-%m-%d"),
        "end_date": dte.strftime("%Y-%m-%d"),
        "timezone": "Asia/Tokyo",
    }
    response = retry_session.get(url, params=params)
    data = response.json()

    logger.info(f"{response.from_cache}, {response.status_code}")
    if response.from_cache:
        return None
    if response.status_code != 200:
        return None

    if type(data) == dict:
        data = [data]

    # データを格納するリスト
    all_forecast_data = []

    # 測定量、時刻、緯度経度の3次元の配列でいいか。
    # dataの構造に沿って、まず緯度経度、物質量x時間で作表する。
    for elem, lon, lat, (x, y) in zip(data, lons, lats, XY):
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(elem["hourly"]["time"][0]),
                periods=len(elem["hourly"]["time"]),
                freq="H",
                tz="Asia/Tokyo",
            ),
            "latitude": lat,
            "longitude": lon,
        } | {item: elem["hourly"][item] for item in OPENMETEO_ITEMS}
        hourly_dataframe = pd.DataFrame(data=hourly_data)
        hourly_dataframe["X"] = x
        hourly_dataframe["Y"] = y
        all_forecast_data.append(hourly_dataframe)

    all_forecast_dataframe = pd.concat(all_forecast_data, ignore_index=True)
    all_forecast_dataframe["ts"] = (
        all_forecast_dataframe["date"].values.astype(np.int64) // 1000000000
    )
    all_forecast_dataframe = all_forecast_dataframe.set_index("ts")
    return all_forecast_dataframe


basicConfig(level=INFO)

zoom = 12  # 14にしてもOpenMeteoのほうがそこまで解像度がない。
# 一旦範囲限定し、OpenMeteoの解像度を探る。
# range_kanagawa = np.array([[139.0, 35.0], [139.4, 35.2]])  # lon,lat
range_kanagawa = np.array([[138.94, 35.13], [139.84, 35.66]])  # lon,lat
XY, shape = tile.tiles(zoom, range_kanagawa)

In [ ]:
from pathlib import Path

dir = f"open-meteo-{zoom}"
Path(dir).mkdir(exist_ok=True)

In [ ]:
import time


logger = getLogger()
for i in range(len(XY)):
    xy = XY[i : i + 1, :]
    logger.info(f"start {dir}/{x}.{y}.{zoom}")
    df = openmeteo_tiles("2009-04-01", "2021-03-31", XY=xy, zoom=zoom)
    logger.info("finish")
    if df is not None:
        x, y = xy[0]
        df = df[(df.X == x) & (df.Y == y)]
        df.to_feather(f"{dir}/{x}.{y}.{zoom}.feather")
        time.sleep(10)

In [ ]:
XY.shape